# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`

 Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге должен формироваться датафрейм вида: `<дата> - <заголовок> - <ссылка>`

### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.  

Итоговый датафрейм формировать со столбцами: `<дата> - <заголовок> - <ссылка> - <текст_статьи>`


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
KEYWORDS = ['python', 'парсинг',]

In [3]:
domen = 'https://habr.com'
path = '/ru/all/'
req = requests.get(domen+path,)
soup = BeautifulSoup(req.text)

In [4]:
articles = soup.find_all(name='article', attrs={'class':'tm-articles-list__item'})

In [5]:
data = []
for article in articles:
    article_title = article.h2.span.text
    article_url = domen + article.h2.a.attrs['href']
    article_dtime = article.find('time').attrs['datetime']
    
    req = requests.get(article_url)
    article_soup = BeautifulSoup(req.text)
    
    text_article = article_soup.find('div',attrs={'id':'post-content-body'}).get_text(separator='\n')
    up_to = text_article.find('Также смотрите')
    text_article = text_article[:up_to]

    for keyword in KEYWORDS:
        if keyword.lower() in text_article.lower():
            data.append([article_dtime, article_title, article_url, text_article])
            break
    time.sleep(0.2)


In [6]:
df = pd.DataFrame(data, columns=['datatime', 'title', 'link', 'text'])
df

,datatime,title,link,text
0,2021-07-22T18:22:45.000Z,Классификатор обращений пользователей (1C + py...,https://habr.com/ru/post/569206/,1. Описание задачи\nВ нашей компании очень мно...
1,2021-07-22T18:22:10.000Z,За что я люблю ассемблер?,https://habr.com/ru/post/569204/,Этой статье уже \nпочти 3 года\n. Однако сегод...
2,2021-07-22T18:21:24.000Z,Зарплаты в Data Science будут расти несмотря н...,https://habr.com/ru/company/skillfactory/blog/...,Оригинал первой части перевода вы найдёте \nзд...
3,2021-07-22T16:32:22.000Z,Работа над ошибками: отчет IT-отдела команды Н...,https://habr.com/ru/post/569176/,21 июля хабра-пользователь \ngrumpysugar\n опу...
4,2021-07-22T15:17:59.000Z,Разбор конкурса IDS Bypass на Positive Hack Da...,https://habr.com/ru/company/pt/blog/569168/,Уже второй раз на конференции Positive Hack Da...
5,2021-07-22T13:49:33.000Z,[Личный опыт] Эстония: почему сюда нанимают мн...,https://habr.com/ru/company/gms/blog/569152/,Эстония не попадает в топы стран для эмиграции...


## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: `<дата утечки> - <источник утечки> - <описание утечки>`  

**Подсказка**: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [7]:
import json
import requests
EMAIL = ['xxx@x.ru', 'yyy@y.com']

In [8]:
url = "https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data"

payload = {"emailAddresses": EMAIL}
headers = {
  'Vaar-Version': '0',
  'Vaar-Header-App-Product': 'hackcheck-web-avast',
  'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
  'Vaar-Header-App-Build-Version': '1.0.0',
  'Content-Type': 'application/json'
}

response = requests.post(url, headers=headers, json=payload)

# print(response.jsonon())


In [9]:
r_dict = response.json()

In [10]:
data = []
for email, v in r_dict['summary'].items():
    breaches = v['breaches']
    for breach in breaches:
#         <дата утечки> - <источник утечки> - <описание утечки>
        breach_info = r_dict['breaches'][str(breach)]
        breach_date = breach_info['publishDate']
        breach_site = breach_info['site']
        breach_desc = breach_info['description']
        
        data.append([email, breach_date, breach_site, breach_desc])

# data

In [11]:
pd.DataFrame(data, columns=['email', 'date', 'site', 'desc'])

,email,date,site,desc
0,xxx@x.ru,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
1,xxx@x.ru,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
2,xxx@x.ru,2016-10-23T00:00:00Z,imesh.com,"In June 2016, a cache of over 51 million user ..."
3,xxx@x.ru,2017-01-31T00:00:00Z,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da..."
4,xxx@x.ru,2017-02-14T00:00:00Z,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
5,xxx@x.ru,2017-02-14T00:00:00Z,parapa.mail.ru,"In July and August 2016, two criminals execute..."
6,yyy@y.com,2016-10-21T00:00:00Z,linkedin.com,"In 2012, online professional networking platfo..."
7,yyy@y.com,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
8,yyy@y.com,2016-10-23T00:00:00Z,imesh.com,"In June 2016, a cache of over 51 million user ..."
9,yyy@y.com,2016-10-24T00:00:00Z,dropbox.com,Cloud storage company Dropbox suffered a major...


### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods
, вам поможет метод [wall.get](https://vk.com/dev/wall.get)  
```
GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ  
```

В итоге должен формироваться датафрейм со столбцами: `<дата поста> - <текст поста>`

In [12]:
GROUP = 'netology'
# TOKEN = 
TOKEN = pd.read_csv('token.csv', index_col=0)['token']['service']

In [13]:
url = 'https://api.vk.com/method/'

In [14]:
method = 'utils.resolveScreenName'
params = {
    'screen_name' : GROUP,
    'access_token' : TOKEN,
    'v' : '5.52'
}

group_id = requests.get(url+method, params=params).json()['response']['object_id']
group_id

30159897

In [15]:
method = 'wall.get'
params = {
    'owner_id' : -group_id,
    'count' : 50,
    'access_token' : TOKEN,
    'v' : '5.52'
}
res = requests.get(url+method, params=params)

In [16]:
# create list of dict with keys 'date' and 'text'
pd.DataFrame([{'date': item['date'], 'text': item['text']} for item in res.json()['response']['items']])

,date,text
0,1625668202,Срочное сообщение от коллеги в пятницу вечером...
1,1626969180,"Часто люди думают, что другие мыслят так же, к..."
2,1626942660,Доставайте попкорн — вышел новый ролик ЭтоИзи ...
3,1626877801,"Даже если вы не дизайнер или иллюстратор, важн..."
4,1626851580,Дата-сайентисты зарабатывают в среднем в два р...
5,1626796860,"Камера, мотор 🎬 \n\nРассказываем, какими навык..."
6,1626769080,"День Нептуна, медляк на дискотеке, страшилки п..."
7,1626709020,Финансисты сегодня стали ключевыми сотрудникам...
8,1626679380,31 июля в 13:00 пройдёт бесплатный онлайн-мита...
9,1626620940,Вот так выглядит профессиональная эволюция Ана...
